In [ ]:
!curl -Ls https://astral.sh/uv/install.sh | sh
!git clone https://github.com/alrazol/GridControl.git
!cd GridControl && uv pip install .

In [ ]:
%%writefile .env
DB_URL="sqlite:///grid.db"
SHOULD_CREATE_TABLES=True
NETWORK_API_BASEURL="http://localhost:8000"
ARTIFACTS_LOCATION="data/experiments"
MLFLOW_TRACKING_URI="sqlite:///mlflow_backend_store.db"
MLFLOW_PORT=8080
LOG_LEVEL=INFO

In [ ]:
import plotly.express as px
import pandas as pd
import json
from pathlib import Path
from datetime import datetime
from src.core.domain.use_cases.import_network_from_json import ETLPipeline
from src.core.infrastructure.settings import Settings
from src.core.infrastructure.adapters.sqlite_network_repository import (
    SQLiteNetworkRepository,
)
from src.core.infrastructure.adapters.network_builder import DefaultNetworkBuilder
from src.core.domain.use_cases.compute_simulated_network import SimulationPipeline
from src.core.infrastructure.services import PyPowsyblCompatService
from src.core.infrastructure.adapters.pypowsybl_loadflow_solver import PyPowSyblLoadFlowSolver
from src.core.constants import LoadFlowType
from src.core.utils import parse_datetime_to_str
from src.core.constants import DEFAULT_TIMEZONE
from src.rl.artifacts.experiment_record import ExperimentRecord
import src.rl.action as action_module
from src.rl.action.enums import DiscreteActionTypes
from src.core.constants import ElementStatus
from src.rl.train import train
from src.rl.agent import DoNothingAgent
from src.rl.config_loaders.agent.config_loader import AgentConfig
from src.core.constants import LoadFlowType
from src.rl.repositories import Repositories
from src.rl.config_loaders.environment.config_loader import EnvironmentConfig
from src.rl.environment import make_env

PATH_TO_LAYOUT = "configs/toy_grid_layout.json"
SIMULATION_CONFIG_PATH = Path("configs/toy_grid_simulation.yaml")
GRID_ID = "toy_grid_layout"
SHOULD_CREATE_TABLE = True
settings = Settings()
network_builder = DefaultNetworkBuilder()

# 1) Grid Layout and Scenario Simulation

## 1.1) Grid Layout

For a start, we want to focus on easily interpretable grids with a friendly JSON format. We therefore implemented a way to define and read from JSON the layout of a grid. In this section we will visualise the raw JSON as we defined it for a very simple grid, made of one load, one generator, and two lines. Agnostic from this file format, we want to read from it and ingest the data into a database. The goal here is to support all kinds of file formats, and have one single repository that is agnostic of the source format where we can then query our grids from. 

In [ ]:
# Visualise the raw JSON we used to define the grid layout

with open(PATH_TO_LAYOUT, "r") as f:
    layout_json = json.load(f)

print(json.dumps(layout_json, indent=4))

In [ ]:
# Ingest the data into the DB repository

network_repository = SQLiteNetworkRepository(
    should_create_tables=SHOULD_CREATE_TABLE,
    db_url=settings.DB_URL,
)

etl_pipeline = ETLPipeline(
    network_repository=network_repository,
    network_builder=network_builder,
)
etl_pipeline.run(file_path=PATH_TO_LAYOUT)

In [ ]:
# Print the elements of the ingested network

net = network_repository.get(network_id=GRID_ID)
net.elements

## 1.2) Simulation 

Once we have the "static" definition of out network, we want to go to a "dynamic" definition, where we simulate some "dynamic" parameters of the network over an arbitrary number of timestamps. In order to achieve that, we define some simulation configuration with some simulation methods that can be used to adjust the trajectory of our elements through time. We first print the simulation yaml configutation, and then run the simulation. The resulting "dynamic" elements (with some timestamp dependent variables) are ingested in the DB repository as well. 

In [ ]:
# Visualise the raw simulation yaml config

with open(SIMULATION_CONFIG_PATH, "r") as f:
    simulation_yaml = f.read()

print(simulation_yaml)

In [ ]:
# Simulate a time series of the network

START = datetime(2025, 1, 1, tzinfo=DEFAULT_TIMEZONE)
END = datetime(2025, 2, 1, tzinfo=DEFAULT_TIMEZONE)

simulation_pipeline = SimulationPipeline(
    config_path=SIMULATION_CONFIG_PATH,
    network_repository=network_repository,
    network_builder=network_builder,
)

simulation_pipeline.apply_pipeline(start=parse_datetime_to_str(START), end=parse_datetime_to_str(END), time_step=1)

In [ ]:
net_simulated = network_repository.get(network_id=f"{GRID_ID}_simulated")

In [ ]:
# Visualise the dynamic parameters for the generator and the load in the network over time

df = pd.concat(
    [
        net_simulated.to_dataframe(element_id="gen1")[["dynamic.Ptarget"]],
        net_simulated.to_dataframe(element_id="gen1")[["static.Pmax"]],
        net_simulated.to_dataframe(element_id="load1")[["dynamic.Pd"]],
    ],
    axis=1,
)
px.line(df)

In [ ]:
# Based on those dynamic parameters, solve a DC load flow for the network at each time step. The solve can potentially fail if the defintions are inconsistent

solver = PyPowSyblLoadFlowSolver(
    to_pypowsybl_converter_service=PyPowsyblCompatService(),
    network_builder=network_builder,
)

net_solved = solver.solve(network=net_simulated, loadflow_type=LoadFlowType.DC)

In [ ]:
# Visualise the flow in the lines, as well as the active power injected by the generator and consumed by the load after the load flow solve
# We notive that as we have two lines active, the power injected by the generator is split across both lines

ELEMENT_ID_1 = "gen1"
ELEMENT_ID_2 = "load1"
line_1_id = "line1"

df_solved= pd.concat(
    [
        net_solved.to_dataframe(element_id=ELEMENT_ID_1)[["solved.p"]].rename(columns={"solved.p": f"solved.p_{ELEMENT_ID_1}"}),
        net_solved.to_dataframe(element_id="load1")[["solved.p"]].rename(columns={"solved.p": f"solved.p_{ELEMENT_ID_2}"}),
        net_solved.to_dataframe(element_id=line_1_id)[["solved.p1"]]
    ],
    axis=1,
)
px.line(df_solved)

# 2) Training an agent

In [ ]:
AGENT_CONFIG_PATH = "GridControl/src/rl/configs/agent/do_nothing_agent.yaml"
ENVIRONMENT_CONFIG_PATH = "GridControl/src/rl/configs/environment/with_outage.yaml"

In [ ]:
# Visualise raw agent yaml config

with open(AGENT_CONFIG_PATH, "r") as f:
    agent_yaml = f.read()
print(agent_yaml)

In [ ]:
# Visualise raw environment yaml config

with open(ENVIRONMENT_CONFIG_PATH, "r") as f:
    environment_yaml = f.read()
print(environment_yaml)

In [ ]:
# Load the configs and setup paths and dependencies

agent_config = AgentConfig.from_yaml(config_path=Path(AGENT_CONFIG_PATH))
environment_config = EnvironmentConfig.from_yaml(
    config_path=Path(ENVIRONMENT_CONFIG_PATH),
)
repositories = Repositories(s=settings)


In [ ]:
# Build the env


env = make_env(
    network_id=f"{GRID_ID}_simulated",
    network_repository=repositories.get_network_repository(),
    environment_config=environment_config,
    loadflow_solver=repositories.get_solver(),
    network_builder=repositories.get_network_builder(),
    network_snapshot_observation_builder=repositories.get_network_snapshot_observation_builder(
        class_name=environment_config.network_snapshot_builder
    ),
    action_space_builder=repositories.get_action_space_builder(),
    one_hot_map_builder=repositories.get_one_hot_map_builder(
        class_name=environment_config.one_hot_map_builder
    ),
    network_observation_handler=repositories.get_network_observation_handler(),
    network_transition_handler=repositories.get_network_transition_handler(
        class_name=environment_config.network_transition_handler
    ),
    loadflow_type=LoadFlowType.DC,
    reward_handler=repositories.get_reward_handler(
        aggregator_name=agent_config.rewards.get("rewards_aggregator"),
        rewards=agent_config.rewards.get("rewards"),
    ),
    action_types=agent_config.action_types,
    observation_memory_length=agent_config.hyperparameters.get(
        "observation_memory_length"
    ),
    outage_handler_builder=repositories.get_outage_handler_builder(),
    network_element_outage_handler_builder=repositories.get_network_element_outage_handler_builder(),
)

## 2.1) DoNothingAgent

The "DoNothingAgent" can't take any action and simply accumulates rewards based on what happends on the grid. By logging the rollout of the episodes they are exposed to, we can get a sense of the dynamics of the environment. Our environment is a stochastic one, where we apply some outages to the lines with some probability distribution that we defined.  

In [ ]:
train(
    experiment_name="do_nothing_agent_toy_grid",
    env=env,
    agent=DoNothingAgent(observation_memory_length=1),
    action_space_builder=repositories.get_action_space_builder(),
    num_episodes=10,
    num_timesteps=100,
    timestep_to_start_updating=20,
    timestep_update_freq=10,
    artifacts_location=settings.ARTIFACTS_LOCATION,
    loss_tracker=repositories.get_loss_tracker(),
    reward_tracker=repositories.get_reward_tracker(),
    log_model=False,
    log_rollout_freq=2,
    registered_model_name=None,
    seed=16,
)

In [ ]:
# Visualise an episode from the training


In [ ]:
# Visualise an RL experiment layout

ROLLOUT_PATH = "src/rl/data/experiments/DQN-agent-exp/4716cdfe142645d99ade9b0207670e6f/artifacts/DQN-agent-exp_rollout_episode_250.json"

with open(Path(ROLLOUT_PATH), "r") as f:
    rl_experiment_layout = json.load(f)

In [ ]:
records = [ExperimentRecord(**i) for i in rl_experiment_layout["records"]]

In [ ]:
px.line(pd.DataFrame([j
 for record in records
 for j in record.next_observation["network_snapshot_observations"][-1]["observations"]
 if j["id"] == "line2"])[["timestamp", "p1"]].set_index("timestamp"))

In [ ]:
def assign_status(status: ElementStatus) -> int:
    if status == ElementStatus.ON:
        return 0
    elif status == ElementStatus.OFF:
        return 1
    elif status == ElementStatus.OUTAGE:
        return 2
    elif status == ElementStatus.MAINTENANCE:
        return 3

df_status = pd.DataFrame([j
 for record in records
 for j in record.next_observation["network_snapshot_observations"][-1]["observations"]
 if j["id"] == "line1"])[["timestamp", "status"]].set_index("timestamp")

df_status["status_code"] = df_status["status"].apply(assign_status)

px.line(df_status[["status_code"]])

In [ ]:
px.line(pd.DataFrame([i.reward for i in records]))

In [ ]:
def assign_action(action: dict, action_type: DiscreteActionTypes) -> int:
    try:
        action_class = getattr(action_module, action_type)
        if action_type == "DoNothingAction":
            action = {}
        if action_type == "SwitchAction":
            action = {"element_id": action["element_id"]}
        if action_type == "StartMaintenanceAction":
            action = {"element_id": action["element_id"]}
        action = action_class(**action)
    except ValueError as e:
        raise ValueError(f"Unknown action type: {action_type}: {e}")
    if isinstance(action, action_module.DoNothingAction):
        return 0
    elif isinstance(action, action_module.SwitchAction) and action.element_id == "line1":
        return 1
    elif isinstance(action, action_module.SwitchAction) and action.element_id == "line2":
        return 2
    elif isinstance(action, action_module.StartMaintenanceAction) and action.element_id == "line1":
        return 3
    elif isinstance(action, action_module.StartMaintenanceAction) and action.element_id == "line2":
        return 4

In [ ]:
px.line(pd.DataFrame([assign_action(i.action, action_type=i.action["action_type"]) for i in records]))